## @yullano90 | Filtros e Exemplo de Cálculo de Proporção

In [ ]:
# Aproveitando o código da Aula 06, vamos adicionar um filtro.
# Alunos com "Voto Obrigatório" cuja UF é "SP" fichãã?

---------------------------------------------------------------------------------------------------

    //Filtragem apenas do público do voto obrigatório em SP:
    FilterVotoObrigatorioSP =
        Table.SelectRows(
            AddC_StatusVotacao, each ([Status_Votação] = "Obrigatório") and ([UF] = "SP")
            )

---------------------------------------------------------------------------------------------------

In [ ]:
# Agora, vamos criar uma função que calcule a proporção de um investimento.
# Para tal vamos criar a função "SomaInvestimento" e depois invocá-la.
# Verifique se o retorno da fução é 11.500. Se sim, avance, tudo certo!
---------------------------------------------------------------------------------------------------

    //Utilizando a função list.sum() passaremos a coluna a ser somada:
    SomaInvestimento = List.Sum(AddC_StatusVotacao[Investimento])

---------------------------------------------------------------------------------------------------
#Alteração no texto Base64 devido a inserção da coluna "Investimento":
'''VY47DoAgEETvQk3BV7CkNTEhUhJuYLTy/s4QTLCYV2ze7G6tIp3PdWshxY
IERCvCAccGGKWUaHKIBpMViYjvIit7AexPtEOK0rNC1REl9+as8hT38olIO
/hP1H4WOebeXujHWcsJgNVe'''

---------------------------------------------------------------------------------------------------

In [ ]:
# Adicione então a coluna "Proporção" com a função SomaInvestimento():

---------------------------------------------------------------------------------------------------

    //Função que faz contagem de linhas, que será o denominador do %:
    AddColumnProporcao =
        Table.AddColumn(
            AddC_StatusVotacao, "Proporção", each (
                [Investimento]/SomaInvestimento * 100
                )
            )

---------------------------------------------------------------------------------------------------


In [ ]:
# Avançando, com o mesma pegada, vamos ver o valor médio distribuído por aluno.
# Vamos então fazer a contagem de alunos com a função abaixo (retorno são 05 linhas):

---------------------------------------------------------------------------------------------------
    //Contagem de alunos.
    TotalAlunos =
        Table.RowCount(
            AddColumnProporcao
            )
---------------------------------------------------------------------------------------------------


In [ ]:
# Espera-se que o código final seja próximo do modelo abaixo:

---------------------------------------------------------------------------------------------------
let
    //Com a adição da coluna "Idade" o texto do Json aumenta ligeiramente:
    Fonte =
        Table.FromRows(Json.Document(Binary.Decompress(Binary.FromText(
            "VY47DoAgEETvQk3BV7CkNTEhUhJuYLTy/s4QTLCYV2ze7G6tIp3PdWshxY
            IERCvCAccGGKWUaHKIBpMViYjvIit7AexPtEOK0rNC1REl9+as8hT38olIO
            /hP1H4WOebeXujHWcsJgNVe", 
            BinaryEncoding.Base64), Compression.Deflate)),
            let _t = ((type nullable text) meta [Serialized.Text = true]) 
            in type table [
                Nome = _t, Nota1 = _t, Nota2 = _t, Frequencia = _t, 
                Idade = _t, UF = _t, Investimento = _t
                ]
            ),

    //Importante normalizar tipos de dados antes de realizar operações matemáticas:
    AlterType =
        Table.TransformColumnTypes(
            Fonte,{
                {"Nome", type text},
                {"Nota1",  type number},
                {"Nota2", type number},
                {"Frequencia",  type number},
                {"Idade", type number},
                {"Investimento", type number}
                }
            ),

    //Criação de uma simples função como exemplo:
    FuncMeanNotes =
        (Nota1, Nota2) => (Nota1 + Nota2) / 2,

    //Coluna adicionando a função "FuncMeanNotes" com o "EACH":
    AddColumnMeanNotes =
        Table.AddColumn(
            AlterType, "Média", each FuncMeanNotes([Nota1],[Nota2])
        ),

    //Coluna de status do aluno, retornando um valor booleano:
    AddAproveColumn =
        Table.AddColumn(
            AddColumnMeanNotes, "Aprovação", each [Frequencia] >=70 and [Média] >= 7
        ),

    //Cálculo do Status para votação:
    FuncStatusVotacao =
        (idade) => if idade < 16 then "Não vota"
        else if idade < 18 then "Facultativo"
        else if idade < 70  then "Obrigatório"
        else "Facultativo",

    //Adicionar uma coluna invocando a função "FuncStatusVotacao":
    AddC_StatusVotacao = Table.AddColumn(
        AddAproveColumn, "Status_Votação", each FuncStatusVotacao([Idade])
        ),


    //Utilizando a função list.sum() passaremos a coluna a ser somada:
    SomaInvestimento = List.Sum(AddC_StatusVotacao[Investimento]),

    //Função que faz contagem de linhas, que será o denominador do %:
    AddColumnProporcao =
        Table.AddColumn(
            AddC_StatusVotacao, "Proporção", each (
                [Investimento]/SomaInvestimento * 100
                )
            )

in
    AddColumnProporcao
    
---------------------------------------------------------------------------------------------------